# Notebook to compute the state evolution for ridge regression and different resampling

**Pour ridge regression, le lambda optimal est constant a $\sigma^2$**

In [1]:
using BootstrapAsymptotics
using Plots
using JSON
using Revise

In [2]:
alpha_range    = exp10.(range(-1.0, stop=3.0, length=50))
lambda         = 0.01
noise_variance = 1.0
max_weight     = 20
rtol           = 1e-20
max_iteration  = 10000

10000

In [ ]:
bootstrap_bootstrap_overlaps = []
bootstrap_bootstrap_hat_overlaps = []

for alpha in alpha_range
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = lambda)
    result = BootstrapAsymptotics.state_evolution(problem, PairBootstrap(p_max = max_weight), PairBootstrap(p_max = max_weight), rtol=rtol, max_iteration=max_iteration)
    push!(bootstrap_bootstrap_overlaps, result[1])
    push!(bootstrap_bootstrap_hat_overlaps, result[2])
end

In [7]:
bootstrap_bootstrap_filename::String = "data/ridge/ridge_bootstrap_bootstrap_overlaps_lambda=$lambda.json"

dict_to_save::Dict = Dict("alpha" => alpha_range,
                          "m"   => [o.m[1] for o in bootstrap_bootstrap_overlaps],
                          "q_diag"   => [o.Q[1, 1] for o in bootstrap_bootstrap_overlaps],
                          "q_off_diag"   => [o.Q[1, 2] for o in bootstrap_bootstrap_overlaps],
                          "v"   => [o.V[1] for o in bootstrap_bootstrap_overlaps],
                          "mhat"=> [o.m[1] for o in bootstrap_bootstrap_hat_overlaps],
                          "qhat_diag"=> [o.Q[1, 1] for o in bootstrap_bootstrap_hat_overlaps],
                          "qhat_off_diag"=> [o.Q[1, 2] for o in bootstrap_bootstrap_hat_overlaps],
                          "vhat"=> [o.V[1] for o in bootstrap_bootstrap_hat_overlaps],
                          )
open(bootstrap_bootstrap_filename, "w") do f
    write(f, JSON.json(dict_to_save))
end
## 


8632

In [8]:
bootstrap_full_overlaps = []
bootstrap_full_hat_overlaps = []

for alpha in alpha_range
    # result = RidgeBootstrapStateEvolution.state_evolution_bootstrap_bootstrap_full(alpha, lambda, noise_variance, max_weight=30, relative_tolerance=1e-20, max_iteration=10000)
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = lambda)
    result = BootstrapAsymptotics.state_evolution(problem, PairBootstrap(p_max = max_weight), FullResampling(), rtol=rtol, max_iteration=max_iteration)
    push!(bootstrap_full_overlaps, result[1])
    push!(bootstrap_full_hat_overlaps, result[2])
end

In [9]:
bootstrap_full_filename::String = "data/ridge/ridge_bootstrap_full_overlaps_lambda=$lambda.json"

dict_to_save::Dict = Dict("alpha" => alpha_range,
                          "q_off_diag" => [o.Q[1, 2] for o in bootstrap_full_overlaps],
                          
                          "qhat_off_diag" => [o.Q[1, 2] for o in bootstrap_full_hat_overlaps],
                          )

open(bootstrap_full_filename, "w") do f
    write(f, JSON.json(dict_to_save))
end

2843

---

# Do full-full correlation 

In [10]:
full_full_overlaps = []
full_full_hat_overlaps = []

for alpha in alpha_range
    # result = RidgeBootstrapStateEvolution.state_evolution_bootstrap_bootstrap_full(alpha, lambda, noise_variance, max_weight=30, relative_tolerance=1e-20, max_iteration=10000)
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = lambda)
    result = BootstrapAsymptotics.state_evolution(problem, FullResampling(), FullResampling(), rtol=rtol, max_iteration=max_iteration)
    push!(full_full_overlaps, result[1])
    push!(full_full_hat_overlaps, result[2])
end

In [11]:
filename::String = "data/ridge/ridge_full_full_overlaps_lambda=$lambda.json"

dict_to_save::Dict = Dict("alpha" => alpha_range,
                          "m"   => [o.m[1] for o in full_full_overlaps],
                          "q_diag"   => [o.Q[1, 1] for o in full_full_overlaps],
                          "q_off_diag"   => [o.Q[1, 2] for o in full_full_overlaps],
                          "v"   => [o.V[1] for o in full_full_overlaps],
                          "mhat"=> [o.m[1] for o in full_full_hat_overlaps],
                          "qhat_diag"=> [o.Q[1, 1] for o in full_full_hat_overlaps],
                          "qhat_off_diag"=> [o.Q[1, 2] for o in full_full_hat_overlaps],
                          "vhat"=> [o.V[1] for o in full_full_hat_overlaps],
                          )

open(filename, "w") do f
    write(f, JSON.json(dict_to_save))
end

7898

# y-resampling correlation

In [12]:
label_label_overlaps = []
label_label_hat_overlaps = []

for alpha in alpha_range
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = lambda)
    result = BootstrapAsymptotics.state_evolution(problem, LabelResampling(), LabelResampling(), rtol=rtol, max_iteration=max_iteration)
    push!(label_label_overlaps, result[1])
    push!(label_label_hat_overlaps, result[2])
end

In [13]:
filename::String = "data/ridge/ridge_label_label_overlaps_lambda=$lambda.json"


dict_to_save::Dict = Dict("alpha" => alpha_range,
                          "m"   => [o.m[1] for o in label_label_overlaps],
                          "q_diag"   => [o.Q[1, 1] for o in label_label_overlaps],
                          "q_off_diag"   => [o.Q[1, 2] for o in label_label_overlaps],
                          "v"   => [o.V[1] for o in label_label_overlaps],
                          "mhat"=> [o.m[1] for o in label_label_hat_overlaps],
                          "qhat_diag"=> [o.Q[1, 1] for o in label_label_hat_overlaps],
                          "qhat_off_diag"=> [o.Q[1, 2] for o in label_label_hat_overlaps],
                          "vhat"=> [o.V[1] for o in label_label_hat_overlaps],
                          )

open(filename, "w") do f
    write(f, JSON.json(dict_to_save))
end

8703

---

# Parametric residual bootstrap

We compute the variance of the residuals, for ridge it's just the training error 

1) On calcule l'overlap de ridge regression 
2) On calcule le bootstrap resampling en utilisant $\rho = q_{\rm erm}$ et $\Delta = \varepsilon_{\rm train}$

In [14]:
function get_train_error(m, q, v, noise_variance; rho = 1.0)
    return (rho + noise_variance + q - 2.0 * m) / (1+v)^2
end

get_train_error (generic function with 1 method)

In [15]:
erm_overlaps = []
residual_residual_overlaps = []
residual_residual_hat_overlaps = []
train_errors = []

for alpha in alpha_range
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = lambda)
    result = BootstrapAsymptotics.state_evolution(problem, FullResampling(), FullResampling(), rtol=rtol, max_iteration=max_iteration)
    push!(erm_overlaps, result[1])
    train_error = get_train_error(result[1].m[1], result[1].Q[1, 1], result[1].V[1], noise_variance, rho = 1.0)
    problem_erm = Ridge(α = alpha, Δ = train_error, ρ = result[1].Q[1, 1], λ = lambda)
    result_2 = BootstrapAsymptotics.state_evolution(problem_erm, LabelResampling(), LabelResampling(), rtol=rtol, max_iteration=max_iteration)
    push!(residual_residual_overlaps,     result_2[1])
    push!(residual_residual_hat_overlaps, result_2[2])
end

In [16]:
to_save = Dict([
    "alpha" => alpha_range,
    "q_diag" => [o.Q[1, 1] for o in residual_residual_overlaps],
    "q_off_diag" => [o.Q[1, 2] for o in residual_residual_overlaps],
    "m" => [o.m[1] for o in residual_residual_overlaps],
])

filename::String = "data/ridge/ridge_residual_residual_overlaps_lambda=$lambda.json"

open(filename, "w") do f
    write(f, JSON.json(to_save))
end

3841

--- 

### Subsampling

In [16]:
# pick = 0.99 for jacknife
proba = 0.8

subsampling_subsampling_overlaps = []
subsampling_subsampling_hat_overlaps = []

for alpha in alpha_range
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = lambda)
    result = BootstrapAsymptotics.state_evolution(problem, Subsampling(r = proba), Subsampling(r = proba), rtol=0.0, max_iteration=100_000)
    push!(subsampling_subsampling_overlaps, result[1])
    push!(subsampling_subsampling_hat_overlaps, result[2])
end

In [17]:
filename::String = "data/ridge/ridge_subsampling_subsampling_overlaps_lambda=$(lambda)_p=$(proba).json"


dict_to_save::Dict = Dict("alpha" => alpha_range,
                          "proba" => proba,
                          "m"   => [o.m[1] for o in subsampling_subsampling_overlaps],
                          "q_diag"   => [o.Q[1, 1] for o in subsampling_subsampling_overlaps],
                          "q_off_diag"   => [o.Q[1, 2] for o in subsampling_subsampling_overlaps],
                          "v"   => [o.V[1] for o in subsampling_subsampling_overlaps],
                          "mhat"=> [o.m[1] for o in subsampling_subsampling_hat_overlaps],
                          "qhat_diag"=> [o.Q[1, 1] for o in subsampling_subsampling_hat_overlaps],
                          "qhat_off_diag"=> [o.Q[1, 2] for o in subsampling_subsampling_hat_overlaps],
                          "vhat"=> [o.V[1] for o in subsampling_subsampling_hat_overlaps],
                          )

open(filename, "w") do f
    write(f, JSON.json(dict_to_save))
end

8634

In [18]:
subsampling_full_overlaps = []
subsampling_full_hat_overlaps = []

for alpha in alpha_range
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = lambda)
    result = BootstrapAsymptotics.state_evolution(problem, Subsampling(r = proba),FullResampling(), rtol=0.0, max_iteration=100_000)
    push!(subsampling_full_overlaps, result[1])
    push!(subsampling_full_hat_overlaps, result[2])
end

In [19]:
filename::String = "data/ridge/ridge_subsampling_full_overlaps_lambda=$(lambda)_p=$(proba).json"


dict_to_save::Dict = Dict("alpha" => alpha_range,
                          "proba" => proba,
                          "m"   => [o.m[1] for o in subsampling_full_overlaps],
                          "q_diag"   => [o.Q[1, 1] for o in subsampling_full_overlaps],
                          "q_off_diag"   => [o.Q[1, 2] for o in subsampling_full_overlaps],
                          "v"   => [o.V[1] for o in subsampling_full_overlaps],
                          "mhat"=> [o.m[1] for o in subsampling_full_hat_overlaps],
                          "qhat_diag"=> [o.Q[1, 1] for o in subsampling_full_hat_overlaps],
                          "qhat_off_diag"=> [o.Q[1, 2] for o in subsampling_full_hat_overlaps],
                          "vhat"=> [o.V[1] for o in subsampling_full_hat_overlaps],
                          )

open(filename, "w") do f
    write(f, JSON.json(dict_to_save))
end

8641

--- 

# Bayes-optimal

In [10]:
bayes_optimal_overlaps = []
bayes_optimal_hat_overlaps = []

for alpha in alpha_range
    # for ridge, b.o is at lambda optimal = noise_variance here
    problem = Ridge(α = alpha, Δ = noise_variance, ρ = 1.0, λ = noise_variance)
    result = BootstrapAsymptotics.state_evolution(problem, FullResampling(), FullResampling(), rtol=rtol, max_iteration=max_iteration)
    push!(bayes_optimal_overlaps, result[1])
    push!(bayes_optimal_hat_overlaps, result[2])
end

In [11]:
filename::String = "data/ridge/ridge_bayes_optimal_overlaps.json"


dict_to_save::Dict = Dict("alpha" => alpha_range,
                          "m"   => [o.m[1] for o in bayes_optimal_overlaps],
                          "q"   => [o.Q[1, 1] for o in bayes_optimal_overlaps],
                          "v"   => [o.V[1] for o in bayes_optimal_overlaps],
                          )

open(filename, "w") do f
    write(f, JSON.json(dict_to_save))
end

3849

--- 

# Compute experimental values for variance and bias

In [3]:
using Colors
using JSON
using Plots
using ProgressMeter
using Statistics
using StableRNGs

In [5]:
α_range_experimental = exp10.(range(-1.0, stop=2.0, length=10))
d = 150

algo_vals = [
    PairBootstrap(; p_max=10), 
    Subsampling(0.8), # 
    Subsampling(0.99), # for Jacknife
    FullResampling(), #
    LabelResampling(),
    ResidualBootstrap(), #
]

rng = StableRNG(0)
K = 100

bias_emp = Dict(algo => fill(NaN, length(α_range_experimental)) for algo in algo_vals)
vars_emp = Dict(algo => fill(NaN, length(α_range_experimental)) for algo in algo_vals)

for algo in algo_vals
    for i in eachindex(α_range_experimental)
        α = α_range_experimental[i]
        n = ceil(Int, d * α)
        problem = Ridge(; λ = lambda, α = α, Δ = 1.0)
        if algo != FullResampling() && algo != LabelResampling()
            bias, var = bias_variance_empirical(rng, problem, algo; n = n, K = K)
        elseif algo == FullResampling()
            bias, var = bias_variance_true(rng, problem; n = n, K, conditional=false)
        elseif algo == LabelResampling()
            bias, var = bias_variance_true(rng, problem; n = n, K, conditional=true)
        end
        bias_emp[algo][i] = bias
        vars_emp[algo][i] = var
    end
end

In [7]:
# save the results 

algo_files = Dict(
    PairBootstrap(; p_max=10) => "data/ridge/exp_ridge_bias_variance_pair_bootstrap_lambda=$lambda.json",
    Subsampling(0.8) => "data/ridge/exp_ridge_bias_variance_subsampling_08_lambda=$lambda.json",
    Subsampling(0.99) => "data/ridge/exp_ridge_bias_variance_subsampling_099_lambda=$lambda.json",
    FullResampling() => "data/ridge/exp_ridge_bias_variance_full_resampling_lambda=$lambda.json",
    LabelResampling() => "data/ridge/exp_ridge_bias_variance_label_resampling_lambda=$lambda.json",
    ResidualBootstrap() => "data/ridge/exp_ridge_bias_variance_residual_bootstrap_lambda=$lambda.json",
)

for algo in algo_vals
    filename = algo_files[algo]
    dict_to_save = Dict(
        "alpha" => α_range_experimental,
        "bias"  => bias_emp[algo],
        "var"   => vars_emp[algo],
        "K"     => K,
    )
    open(filename, "w") do f
        write(f, JSON.json(dict_to_save))
    end
end

--- 

### Compute the Bayes-optimal overlap (variance = 1 - q in the state evolutions)

In [3]:
bayes_optimal_q = fill(0.0, length(alpha_range))

for i in eachindex(alpha_range)
    α = alpha_range[i]
    problem = Ridge(α = α, Δ = noise_variance, ρ = 1.0, λ = 1.0)
    res     = state_evolution_BayesOpt(problem, rtol=1e-8, max_iteration=100)
    bayes_optimal_q[i] = res.q
end

(overlaps = Overlaps([0.048750780379935736, 0.048750780379935736], [0.04875078060516319 0.002376638587652727; 0.002376638587652727 0.04875078060516319], [0.9512492196200644 0.0; 0.0 0.9512492196200644]), hatoverlaps = HatOverlaps([0.05124921972779644, 0.05124921972779644], [0.05124921973371199 0.0; 0.0 0.05124921973371199], [0.05124921972779644 0.0; 0.0 0.05124921972779644]), stats = (

converged = true, nb_iterations = 8))
(overlaps = Overlaps([0.058520850426761986, 0.058520850426761986], [0.05852085044505012 0.0034246899346714486; 0.0034246899346714486 0.05852085044505012], [0.941479149573238 0.0; 0.0 0.941479149573238]), hatoverlaps = HatOverlaps([0.06215841364583272, 0.06215841364583272], [0.06215841364641825 0.0; 0.0 0.06215841364641825], [0.06215841364583272 0.0; 0.0 0.06215841364583272]), stats = (converged = true, nb_iterations = 9))
(overlaps = Overlaps([0.07016974040673465, 0.07016974040673465], [0.07016974046808847 0.00492379246874853; 0.00492379246874853 0.07016974046808847], [0.9298302595932653 0.0; 0.0 0.9298302595932653]), hatoverlaps = HatOverlaps([0.07546510737209533, 0.07546510737209533], [0.07546510737449454 0.0; 0.0 0.07546510737449454], [0.07546510737209533 0.0; 0.0 0.07546510737209533]), stats = (converged = true, nb_iterations = 9))
(overlaps = Overlaps([0.08402190206493813, 0.08402190206493813], [0.08402190226627955 0.007059680026610054; 0.0070

, stats = (converged = true, nb_iterations = 10))
(overlaps = Overlaps([0.9249931769487588, 0.9249931769487588], [0.9249931771608779 0.855612377401758; 0.855612377401758 0.9249931771608779], [0.07500682305124115 0.0; 0.0 0.07500682305124115]), hatoverlaps = HatOverlaps([12.332120477405729, 12.332120477405729], [12.332120479839078 0.0; 0.0 12.332120479839078], [12.332120477405729 0.0; 0.0 12.332120477405729]), stats = (converged = true, nb_iterations = 10))
(overlaps = Overlaps([0.9377367956563386, 0.9377367956563386], [0.9377367967336863 0.8793502979278178; 0.8793502979278178 0.9377367967336863], [0.06226320434366133 0.0; 0.0 0.06226320434366133]), hatoverlaps = HatOverlaps([15.060850390601264, 15.060850390601264], [15.060850405875973 0.0; 0.0 15.060850405875973], [15.060850390601264 0.0; 0.0 15.060850390601264]), stats = (converged = true, nb_iterations = 9))
(overlaps = Overlaps([0.948343461992572, 0.948343461992572], [0.9483434623335654 0.8993553219040568; 0.8993553219040568 0.94834

In [13]:
filename = "data/ridge/ridge_bayes_optimal_overlaps.json"

dict_to_save = Dict("alpha" => alpha_range,
                    "q" => bayes_optimal_q
                    )

open(filename, "w") do f
    write(f, JSON.json(dict_to_save))
end

1171